# **Car price prediction using the dataset 'Car Features and MSRP'**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
car=pd.read_csv('/kaggle/input/cardataset/data.csv')

In [ ]:
car.head()

# **Finding and cleaning missing values**

In [ ]:
allna = (car.isnull().sum() / len(car))*100
allna = allna.drop(allna[allna == 0].index).sort_values()
plt.figure(figsize=(8, 4))
allna.plot.barh(color=('red', 'black'), edgecolor='black')
plt.title('Missing values percentage per column', fontsize=15, weight='bold' )
plt.xlabel('Percentage', weight='bold', size=15)
plt.ylabel('Features with missing values', weight='bold')
plt.yticks(weight='bold')
plt.show()

In [ ]:
NA=car[['Engine Fuel Type','Engine HP', 'Engine Cylinders', 'Number of Doors', 'Market Category']]
NAcat=NA.select_dtypes(include='object')
NAnum=NA.select_dtypes(exclude='object')
print(NAcat.shape[1],'categorical features with missing values')
print(NAnum.shape[1],'numerical features with missing values')

In [ ]:
NAnum.head()

Cleaning numerical feature with filling forward method

In [ ]:
car['Engine HP']=car['Engine HP'].fillna(method='ffill')
car['Engine Cylinders']=car['Engine Cylinders'].fillna(method='ffill')
car['Number of Doors']=car['Number of Doors'].fillna(method='ffill')

In [ ]:
NAcat.head()

Cleaning 'Engine Fuel' using fill forward method

In [ ]:
car['Engine Fuel Type']=car['Engine Fuel Type'].fillna(method='ffill')

In [ ]:
plt.style.use('seaborn')
fig= plt.figure(figsize=(16,10))

market=car['Market Category'].value_counts().head(5).to_frame()
m= market.style.background_gradient(cmap='Blues')
m

Most of the cars belong to the Market Category " Crossover" 

Cleaning 'Market Category' by filling it with 'Luxury' assuming that most of the cars belong to the Market category " Crossover".


In [ ]:
car['Market Category']=car['Market Category'].fillna('Crossover')

In [ ]:
car.isnull().sum().sort_values(ascending=False).head()

#  Data Exploration & Visualization

### **Checking most expensive brands**


In [ ]:
modelp = car.groupby(['Make']).sum()[['MSRP','Popularity','Engine HP','Engine Cylinders','Number of Doors']].nlargest(10, 'MSRP')
modelp.groupby(['Make']).sum()['MSRP'].nlargest(10).iplot(kind='bar', xTitle='Make', yTitle='MSRP',title='Top 10 expensive Car Brands')

Inference : Chevrolet has highest MSRP

In [ ]:
Chev=car[car['Make'].str.contains('Chevrolet')]
ch=Chev.sort_values(by=['MSRP'],ascending=False).nlargest(5, 'MSRP')
chevmodel= ch.style.background_gradient(cmap='Greens')
chevmodel

Inference : The chevrolet models of all 3 years have the same features except for the Vehicle Style: Convertible Corvette are worth more than Coupe Corvette.

### **Relation between vehicle style and MSRP**

In [ ]:
sns.set({'figure.figsize':(20,10)})
VS=sns.barplot(x=car['MSRP'], y=car['Vehicle Style'])
plt.title('Vehicle Style and MSRP', weight='bold', fontsize=18)
plt.xlabel('MSRP', weight='bold',fontsize=14)
plt.ylabel('Vehicle Style', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

Inference : Coupe,Convertible,Sedan,4dr SUV makes the model worth more

### Checking least expensive cars

In [ ]:
modelp1 = car.groupby(['Make']).sum()[['MSRP','Popularity','Engine HP','Engine Cylinders','Number of Doors']].nsmallest(10, 'MSRP')
modelp1.groupby(['Make']).sum()['MSRP'].nlargest(10).iplot(kind='bar', xTitle='Make', yTitle='MSRP',
                                                                     title='Top 10 least expensive Car Brands')

In [ ]:
F=car[car['Make'].str.contains('FIAT')]
FI=F.sort_values(by=['MSRP'],ascending=False).nsmallest(5, 'MSRP')
Fiat= FI.style.background_gradient(cmap='Oranges')
Fiat

Choosing FIAT models <p>
    Inference : 2dr Hatchback makes the model worth less

### Correlation between features

In [ ]:
car_corr=car.corr()
sns.heatmap(car_corr, cmap='viridis')
plt.title("Correlation between features", 
          weight='bold', 
          fontsize=18)
plt.show()

Inference : 
Positive correlation betweeen 
* Engine HP and Engine Cylinders
* Engine HP and MSRP
* Engine Cylinders and MSRP
Negative correlation between
* Engine HP and Highway MPG

### Relation between Engine Cylinders, Engine HP, Price

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(x=car['Engine Cylinders'], y=car['Engine HP'], color='maroon', alpha=0.7)
plt.title('Engine Cylinders on Engine HP', weight='bold', fontsize=18)
plt.xlabel('Engine Cylinders', weight='bold',fontsize=14)
plt.ylabel('Engine HP', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x=car['Engine HP'], y=car['MSRP'], color='red')
plt.title('Engine HP on MSRP', weight='bold', fontsize=18)
plt.xlabel('MSRP', weight='bold',fontsize=14)
plt.ylabel('Engine HP', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x=car['Engine Cylinders'], y=car['MSRP'], color='red')
plt.title('Engine Cylinders on MSRP', weight='bold', fontsize=18)
plt.xlabel('MSRP', weight='bold',fontsize=14)
plt.ylabel('Engine Cylinders', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

Inference : An engine with more cylinders produces more power, and more power means a high MSRP.

### Correlation between MPG, Engine HP, Price

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x=car["Engine HP"], y=car["highway MPG"], line_kws={"color":"red","alpha":1,"lw":5})
plt.title('Highway MPG and Engine HP', weight='bold', fontsize=18)
plt.xlabel('Engine HP', weight='bold',fontsize=14)
plt.ylabel('Highway MPG', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x=car["Engine HP"], y=car["city mpg"], line_kws={"color":"red","alpha":1,"lw":5})
plt.title('City MPG and Engine HP', weight='bold', fontsize=18)
plt.xlabel('Engine HP', weight='bold',fontsize=14)
plt.ylabel('City MPG', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x=car["MSRP"], y=car["highway MPG"], line_kws={"color":"red","alpha":1,"lw":5})
plt.title('Highway MPG and MSRP', weight='bold', fontsize=18)
plt.xlabel('MSRP', weight='bold',fontsize=14)
plt.ylabel('Highway MPG', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x=car["MSRP"], y=car["city mpg"], line_kws={"color":"red","alpha":1,"lw":5})
plt.title('City MPG and MSRP', weight='bold', fontsize=18)
plt.xlabel('MSRP', weight='bold',fontsize=14)
plt.ylabel('City MPG', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

Inference :
Negative correlation between highway and city mpg and the Engine HP.

### Correlation between MSRP and other features

In [ ]:
plt.style.use('seaborn')
fig= plt.figure(figsize=(16,10))
#first row, first col
ax1 = plt.subplot2grid((2,2),(0,0))
sns.boxplot(x=car['Number of Doors'], y=car['MSRP'],color='Red')

#first row sec col
ax1 = plt.subplot2grid((2,2), (0, 1))
plt.scatter(x=car['Driven_Wheels'], y=car['MSRP'], color='Orange')
plt.ylabel('MSRP')
plt.title('Driven Wheels')

#Second row first column
ax1 = plt.subplot2grid((2,2), (1, 0))
sns.barplot(x=car['Transmission Type'], y=car['MSRP'])
plt.xticks(rotation=35)

#second row second column
ax1 = plt.subplot2grid((2,2), (1, 1))
sns.barplot(x=car['Vehicle Size'], y=car['MSRP'])

plt.yticks(weight='bold')

plt.show()

Inference :
Features that contribute to high MSRP
1. Number of doors : 2
2. Driven Wheels : All
3. Transmission : Automated
4. Vehicle size : Large

### Relation between popularity and other features

In [ ]:
modelp = car.groupby(['Make']).sum()[['MSRP','Popularity','Engine HP','Engine Cylinders','Number of Doors']].nlargest(5, 'Popularity')
modelp.groupby(['Make']).sum()['Popularity'].nlargest(10).iplot(kind='bar', xTitle='Make', yTitle='Popularity',title='Top 5 popular Car Brands')

In [ ]:
#F=car.loc[car['Make'] == 'BMW']
F=car[car['Make'].str.contains('Ford')]
Ford=F.sort_values(by=['Popularity'],ascending=False).nlargest(10, 'MSRP')
Ford= Ford.style.background_gradient(cmap='Reds')
Ford

In [ ]:
sns.regplot(x=car["MSRP"], y=car["Popularity"], line_kws={"color":"red","alpha":1,"lw":5})
plt.title('Popularity and MSRP', weight='bold', fontsize=18)
plt.xlabel('MSRP', weight='bold',fontsize=14)
plt.ylabel('Popularity', weight='bold', fontsize=14)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

### Comparing Engine Fuel Type

In [ ]:
EFT=car[['Engine Fuel Type',
      'Engine HP',
      'Engine Cylinders',
      'highway MPG',
         'MSRP',
        'city mpg']].groupby(['Engine Fuel Type']).agg('median').sort_values(by=['Engine HP'],ascending=False)
EFT1= EFT.style.background_gradient(cmap='Purples')
EFT1

### Conclusion <p>
* High numbers of Engine Cylinders produce Higher Engine Horsepower.
* High Engine Horsepower means a High MSRP and a low MPG value.
* Cars with : 2 doors - all wheel drive - large size - Automated manual transmission size and Convertible Style mark a high MSRP Values.